<a href="https://colab.research.google.com/github/kikansha-Tomasu/dl_lecture_competition_pub/blob/event-camera-competition/%E6%9C%80%E7%B5%82%E8%AA%B2%E9%A1%8C_DL%E5%9F%BA%E7%A4%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/dl_lecture_competition_pub/data/train.zip', '/content/drive/MyDrive/dl_lecture_competition_pub/data')
shutil.unpack_archive('/content/drive/MyDrive/dl_lecture_competition_pub/data/test.zip', '/content/drive/MyDrive/dl_lecture_competition_pub/data')

In [ ]:
!git clone https://github.com/kikansha-Tomasu/dl_lecture_competition_pub.git

Cloning into 'dl_lecture_competition_pub'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Receiving objects: 100% (52/52), 61.98 KiB | 15.50 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [10]:
!git config --global user.email "ryosktkg@gmail.com"
!git config --global user.name "kikansha-Tomasu"

In [3]:
%pwd

'/content'

In [3]:
!git config --global init.defaltBranch event-camera-competition

In [4]:
%cd /content/drive/MyDrive/dl_lecture_competition_pub
!git checkout event-camera-competition

/content/drive/MyDrive/dl_lecture_competition_pub
M	configs/base.yaml
M	main.py
Already on 'event-camera-competition'
Your branch is up to date with 'origin/event-camera-competition'.


In [5]:
!pip install hydra-core --upgrade

In [6]:
!pip install hdf5plugin

In [7]:
!pip install -r requirements.txt

In [ ]:
%env HYDRA_FULL_ERROR=1
!python main.py #learning_rate = 0.01

env: HYDRA_FULL_ERROR=1
on epoch: 1
  0% 0/252 [00:00<?, ?it/s]batch 0 loss: 195.50885279381794
  0% 1/252 [01:51<7:46:48, 111.59s/it]batch 1 loss: 244.5779246527539
  1% 2/252 [03:35<7:26:09, 107.08s/it]batch 2 loss: 211.7508325378219
  1% 3/252 [05:21<7:21:43, 106.44s/it]batch 3 loss: 203.7842623451583
  2% 4/252 [07:03<7:13:36, 104.90s/it]batch 4 loss: 114.08403948476143
  2% 5/252 [08:43<7:04:05, 103.02s/it]batch 5 loss: 96.89662105995001
  2% 6/252 [10:24<7:00:08, 102.48s/it]batch 6 loss: 70.84496936379614
  3% 7/252 [12:05<6:56:24, 101.98s/it]batch 7 loss: 45.01987555964849
  3% 8/252 [13:45<6:51:20, 101.15s/it]batch 8 loss: 79.35438949542758
  4% 9/252 [15:27<6:51:00, 101.48s/it]batch 9 loss: 50.50680220892743
  4% 10/252 [17:11<6:51:59, 102.15s/it]batch 10 loss: 45.07868989968732
  4% 11/252 [18:54<6:52:17, 102.65s/it]batch 11 loss: 45.891296070105035
  5% 12/252 [20:36<6:49:15, 102.32s/it]batch 12 loss: 45.93447280720821
  5% 13/252 [22:16<6:44:54, 101.65s/it]batch 13 loss: 38

In [ ]:
%env HYDRA_FULL_ERROR=1
!python main.py #ZCA白色化あり, train/batch_size = 4

env: HYDRA_FULL_ERROR=1
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
python3: can't open file 'main.py': [Errno 107] Transport endpoint is not connected


In [13]:
!git remote set-url origin https://kikansha-Tomasu:201209Waka/kikansha-Tomasu/dl_lecture_competition_pub.git
!git push origin master

error: src refspec master does not match any
error: failed to push some refs to 'https://kikansha-Tomasu:201209Waka/kikansha-Tomasu/dl_lecture_competition_pub.git'


In [ ]:
"""
メモリ不足解消のため、configs/base.yamlのtrainのbatch_sizeを16から8に変更
データ拡張のために、ZCA白色化を施した
ZCA白色化にバッチ処理を施した
"""
#元のmain.py
import torch
import hydra
from omegaconf import DictConfig
from torch.utils.data import DataLoader
import random
import numpy as np
from src.models.evflownet import EVFlowNet
from src.datasets import DatasetProvider
from enum import Enum, auto
from src.datasets import train_collate
from tqdm import tqdm
from pathlib import Path
from typing import Dict, Any
import os
import time


class RepresentationType(Enum):
    VOXEL = auto()
    STEPAN = auto()

def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

def compute_epe_error(pred_flow: torch.Tensor, gt_flow: torch.Tensor):
    '''
    end-point-error (ground truthと予測値の二乗誤差)を計算
    pred_flow: torch.Tensor, Shape: torch.Size([B, 2, 480, 640]) => 予測したオプティカルフローデータ
    gt_flow: torch.Tensor, Shape: torch.Size([B, 2, 480, 640]) => 正解のオプティカルフローデータ
    '''
    epe = torch.mean(torch.mean(torch.norm(pred_flow - gt_flow, p=2, dim=1), dim=(1, 2)), dim=0)
    return epe

def save_optical_flow_to_npy(flow: torch.Tensor, file_name: str):
    '''
    optical flowをnpyファイルに保存
    flow: torch.Tensor, Shape: torch.Size([2, 480, 640]) => オプティカルフローデータ
    file_name: str => ファイル名
    '''
    np.save(f"{file_name}.npy", flow.cpu().numpy())

@hydra.main(version_base=None, config_path="configs", config_name="base")
def main(args: DictConfig):
    set_seed(args.seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    '''
        ディレクトリ構造:

        data
        ├─test
        |  ├─test_city
        |  |    ├─events_left
        |  |    |   ├─events.h5
        |  |    |   └─rectify_map.h5
        |  |    └─forward_timestamps.txt
        └─train
            ├─zurich_city_11_a
            |    ├─events_left
            |    |       ├─ events.h5
            |    |       └─ rectify_map.h5
            |    ├─ flow_forward
            |    |       ├─ 000134.png
            |    |       |.....
            |    └─ forward_timestamps.txt
            ├─zurich_city_11_b
            └─zurich_city_11_c
        '''

    # ------------------
    #    Dataloader
    # ------------------
    loader = DatasetProvider(
        dataset_path=Path(args.dataset_path),
        representation_type=RepresentationType.VOXEL,
        delta_t_ms=100,
        num_bins=4
    )
    train_set = loader.get_train_dataset()
    test_set = loader.get_test_dataset()
    collate_fn = train_collate
    train_data = DataLoader(train_set,
                                 batch_size=args.data_loader.train.batch_size,
                                 shuffle=args.data_loader.train.shuffle,
                                 collate_fn=collate_fn,
                                 drop_last=False)
    test_data = DataLoader(test_set,
                                 batch_size=args.data_loader.test.batch_size,
                                 shuffle=args.data_loader.test.shuffle,
                                 collate_fn=collate_fn,
                                 drop_last=False)

    '''
    train data:
        Type of batch: Dict
        Key: seq_name, Type: list
        Key: event_volume, Type: torch.Tensor, Shape: torch.Size([Batch, 4, 480, 640]) => イベントデータのバッチ
        Key: flow_gt, Type: torch.Tensor, Shape: torch.Size([Batch, 2, 480, 640]) => オプティカルフローデータのバッチ
        Key: flow_gt_valid_mask, Type: torch.Tensor, Shape: torch.Size([Batch, 1, 480, 640]) => オプティカルフローデータのvalid. ベースラインでは使わない

    test data:
        Type of batch: Dict
        Key: seq_name, Type: list
        Key: event_volume, Type: torch.Tensor, Shape: torch.Size([Batch, 4, 480, 640]) => イベントデータのバッチ
    '''
    # ------------------
    #       Model
    # ------------------
    model = EVFlowNet(args.train).to(device)

    # ------------------
    #   optimizer
    # ------------------
    optimizer = torch.optim.Adam(model.parameters(), lr=args.train.initial_learning_rate, weight_decay=args.train.weight_decay)
    # ------------------
    #   Start training
    # ------------------
    model.train()
    for epoch in range(args.train.epochs):
        total_loss = 0
        print("on epoch: {}".format(epoch+1))
        for i, batch in enumerate(tqdm(train_data)):
            batch: Dict[str, Any]
            event_image = batch["event_volume"].to(device) # [B, 4, 480, 640]
            ground_truth_flow = batch["flow_gt"].to(device) # [B, 2, 480, 640]
            flow = model(event_image) # [B, 2, 480, 640]
            loss: torch.Tensor = compute_epe_error(flow, ground_truth_flow)
            print(f"batch {i} loss: {loss.item()}")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_data)}')

    # Create the directory if it doesn't exist
    if not os.path.exists('checkpoints'):
        os.makedirs('checkpoints')

    current_time = time.strftime("%Y%m%d%H%M%S")
    model_path = f"checkpoints/model_{current_time}.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

    # ------------------
    #   Start predicting
    # ------------------
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    flow: torch.Tensor = torch.tensor([]).to(device)
    with torch.no_grad():
        print("start test")
        for batch in tqdm(test_data):
            batch: Dict[str, Any]
            event_image = batch["event_volume"].to(device)
            batch_flow = model(event_image) # [1, 2, 480, 640]
            flow = torch.cat((flow, batch_flow), dim=0)  # [N, 2, 480, 640]
        print("test done")
    # ------------------
    #  save submission
    # ------------------
    file_name = "submission"
    save_optical_flow_to_npy(flow, file_name)

if __name__ == "__main__":
    main()
